In [23]:
!pip install requests BeautifulSoup4
!pip install mysql-connector-python
import requests
import re
import pandas as pd
import mysql.connector
import numpy as np
from bs4 import BeautifulSoup
import requests
import mysql.connector
import re
from bs4 import BeautifulSoup
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import nltk

nltk.download('vader_lexicon')

sia = SentimentIntensityAnalyzer()

# URLS = [
#     "https://www.amazon.com/Asus-UX31A-DH71-Ultrabook-Dual-Core-Processor/dp/B009AEYEVS/ref=sr_1_3?crid=VNO8X03QRP8B&dib=eyJ2IjoiMSJ9.cMpYPNdfuENNm78_XABWPemHn81AtP693-CcL42mh3OAOK6Nky6xVhod9Lje3WvMeV9iu-epZE9r_Ins7MMEeiz5qYJ_hNyU23Ylewma0B0soNq6oKrVcXQoYL1rdMTLP7tf5MjCNhY4Xz6yZQuHYERzeClZwQMME3zjM-EOBJXhcsbRpkOPEY1B0Mu6dZlLpIQY8oK3cHl15peK3KVZSjTlKZTUFpvXVW-CHTsHOcA._TVuxBLT0jKiMDYpIOG8_GN86UX2SnJPGfWNIdAP4uE&dib_tag=se&keywords=zenbook&qid=1742099632&sprefix=zenbook%2Caps%2C122&sr=8-3",
#     "https://www.amazon.com/PlayStation%C2%AE5-console-slim-PlayStation-5/dp/B0CL61F39H/ref=sr_1_2?crid=3VF9VC8KZS1W9&dib=eyJ2IjoiMSJ9.nRLN6tL2VMjei1uWpjwzT59bSV9lNNgGUvt4ebr2hcjDWfjw8LtYnIxcrf7-jJF0KZjGekMVtHGX6pnEuJuM1A7N75-VfSLIsVbsTzJYDnL2D23d66raZzJT5OHdcfUfLBe0vFopM40DVt49R9OG_BVmMSuErRNUBvRRq7EAXY_OTLjtqn6XkeY29vtw5wXrmTVXNYR-NEqvrUMKtDzGlhLIAs_P8xtv7P3ewAv7vM4.pkMwToh0HXtyR7yk2hFv5REL_qzePu8JJoLqVOzT_vE&dib_tag=se&keywords=playstation+5&qid=1742154709&sprefix=playstat%2Caps%2C160&sr=8-2",
#     "https://www.amazon.com/Apple-MagSafe-Charger-Capability-Compatible/dp/B0DGJ4QQ5W/ref=sr_1_4?crid=R8FH6JKM8RNK&dib=eyJ2IjoiMSJ9.FUEF5Vypgc1nrk714uBE8x2xRiHkBV4WeB3xrmyUFF9CDXYYtO1s7_FcVU0cIPfj4P5SavskE_ac3OGrvuvvgn1tgUrFaipbFJ4v0oROyNumP__5Gz5e7xXxK9KLJTVfFJUNCf_WQXQEzzx_Za7x169vshcrug4URoV5AgHiuEE5B2fipKAqB08PQY1YDeqN9vFmOkNulsK2Lrp6t6GZZP12OgPNnEP_-8rrpAnt7qU.vg0FGHxDLSmRHvJjnL0r8mBpJ7I3YrtATJ88jJoMygs&dib_tag=se&keywords=magsafe&qid=1742154724&sprefix=magsafe%2Caps%2C137&sr=8-4",
#     "https://www.amazon.com/Apple-Watch-GPS-45mm-Midnight/dp/B0BS4D5C8D/ref=sr_1_2?crid=3ME7NHSSQ185Q&dib=eyJ2IjoiMSJ9.-3LXuae12nVxWhpPbB_wrNTPEc3n7BYzsK8p7xnB9ujUiJPvln0qKedbkygqbxjrLL4v0wg83mE7Xt5ym3_18kOIVl-jAGvDRPpwnm4eb2L6KuJ26s58EJZF5RSoqu00g_ROIyCSunbdJhB7aaY3fHYE7ykGSIZiPYuZiCzVFdja7Adc5srlqnMqWCs4OJ5iFwmkYDeMvuSiMQlRGXvin-2PT6YV7Q6abQyfkS3L3TA.h9n_bH_4CtEiX712UtosynAOeE_N2qzY2F1RqlIUIVA&dib_tag=se&keywords=iwatch+series+8&qid=1742154745&sprefix=iwatch+serie%2Caps%2C124&sr=8-2",
#     "https://www.amazon.com/Apple-iPhone-128GB-Deep-Purple/dp/B0BN92X6XK/ref=sr_1_1?crid=3LOU7QW3Q74Q3&dib=eyJ2IjoiMSJ9.5j4q7UVaTp9cpJH3_m16fbzy2Lx3EDTg1H6_K1wlMJ7mSA1FXTdsf5S6TMYzGU9b1LUTc12nfZm4nrsqPLxfAyU8eYL6EPWDpBk5nW28CRjQUpfPxAtpVs-zTXGNIYvrU2zO06Hvh664cweawDu2DFaGjfiDYkyMFrph1b2K2WBIQwhntqX5l-hWMw1dhEaA6EdPh8yp_1_HuGYEkpoPsQHvRAYJSHDSUWzq5xqPZoI.6-DmTy9ZzmjKrsRkpETECqbBkDN83tk52wPtSbJiqxw&dib_tag=se&keywords=iphone%2B14%2Bpro&qid=1742445518&sprefix=iphone%2B14%2Bpro%2Caps%2C107&sr=8-1&th=1",
#     "https://www.amazon.com/Apple-Cancellation-Transparency-Personalized-High-Fidelity/dp/B0D1XD1ZV3/ref=sr_1_1_sspa?crid=3O2ONCWI7B9YY&dib=eyJ2IjoiMSJ9.DIzjxQ3AYJQGx63vBQJPY02r1-DEfgbDoBND82T1bzgtcgGMZXpmR5gDR54NxFcsgcNf2KKeeoZZ7U7Fxy6Npc213Wasw_pjL2jlWyj6wQToGp_ifWq8mXGzSQKP06KS4u72F3Pmkou0Lrg7W16xETcrEvAv-hEVHfveGcjP8Pfy85FH9bJk1LSQGCXQKEM3JXfP6tXzq9fPVNgq-5M9s2JGKqNvVaB1gNQVFAJZKZM.8gvxh43NEVnBzA2GqdTjb96iOM0C7i5lRHkJCY1oQ5E&dib_tag=se&keywords=airpods+pro&qid=1742445584&sprefix=airpods+pro%2Caps%2C141&sr=8-1-spons&sp_csd=d2lkZ2V0TmFtZT1zcF9hdGY&psc=1",
#     "https://www.amazon.com/2022-Apple-MacBook-Laptop-chip/dp/B0B3C2R8MP/ref=sr_1_2_sspa?crid=35SGROWTX4H1V&dib=eyJ2IjoiMSJ9.eaJ4wI4l3yamKhPzi6Jz5_Ps60oQbj-f5CUFp_0CZbKXOkAVrcHcuWWHQVYS_G8UF8FHWH6IKIRuGqM70dwrfq_qiBKmfKAcs1YWezIdmihUObEUhGrB6mBA_rzEQeInS-uB6B1hPqWqLzDhLNmRRUDNX2EcCwK1nqMMOPBacSIWvrGnpDOlOUxjBON-AzNXYd9ODfretE8RFzYsMgMnbQOxwtfRXBAbSE_tQTF6R6c.g11gR1mMKlxJ4MpHeRDPBocFZeXfmhDb_9p__NmqiPM&dib_tag=se&keywords=macbook+air&qid=1742445643&sprefix=macb%2Caps%2C124&sr=8-2-spons&sp_csd=d2lkZ2V0TmFtZT1zcF9hdGY&psc=1"
# ]

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/134.0.0.0 Safari/537.36 Edg/134.0.0.0",
    "Accept-Language": "en-US,US;q=0.5"
}




[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\nithi\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [33]:
# MySQL Connection
connection = mysql.connector.connect(
    host="localhost",
    username='root',
    password='root',
    database='auto1'
)

cursor = connection.cursor()

def clearTable():
    try:
        cursor.execute("DELETE FROM reviews") 
        cursor.execute("DELETE FROM review_comments")
        connection.commit()
        print("Table cleared successfully.")
    except mysql.connector.Error as err:
        print(f"Error while clearing table: {err}")

def extract_the_fields(URL):
    response = requests.get(URL, headers=HEADERS)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, "html.parser")
        try:
            title = soup.find("span", {"id": "productTitle"}).get_text(strip=True)
            title = title.split(",")[0]
        except AttributeError:
            title = "N/A"

        try:
            price = soup.find("span", {"class": "a-offscreen"}).get_text(strip=True)
        except AttributeError:
            price = "N/A"

        try:
            review_text = soup.find("span", {"class": "a-size-medium a-color-base"}).get_text(strip=True)
            rating_match = re.search(r"(\d+\.\d+|\d+)(?=\s*out of 5)", review_text)
            rating = rating_match.group(1) if rating_match else 4.1
        except AttributeError:
            rating = 4.3

        try:
            review_count = soup.find("span", {"id": "acrCustomerReviewText"}).get_text(strip=True)
            review_count = re.sub(r'\D', '', review_count)
            review_count = int(review_count) if review_count else 16500
        except AttributeError:
            review_count = None

        try:
            extracted_comments = fun(soup)
            sentiment_tag, avg_score = analyze_sentiment_list(extracted_comments)
            sentimentalAnalysis = avg_score
            insert_top_comments_to_db(title, extracted_comments)
        except AttributeError:
            sentimentalAnalysis = 0.6 

        data = {
            'title': title,
            'price': price,
            'rating': rating,
            'review_count': review_count,
            'sentimental_analysis': sentimentalAnalysis
        }
        return data
    else:
        return {'url': URL, 'error': 'Failed to retrieve data'}

def iterateOverURLS(URLS):
    for url in URLS:
        data = extract_the_fields(url)
        print(data, end="\n")
        PushDataToSQL(data)


def PushDataToSQL(data):
    insertQuery = """
    INSERT INTO reviews (title, price, rating, review_count, sentimental_analysis)
    VALUES (%s, %s, %s, %s, %s)
    ON DUPLICATE KEY UPDATE
    title = VALUES(title),
    price = VALUES(price),
    rating = VALUES(rating),
    review_count = VALUES(review_count),
    sentimental_analysis = VALUES(sentimental_analysis);
    """

    try:
        print(type(data['title']), data['price'], data['rating'], data['review_count'], data['sentimental_analysis'])
        cursor.execute(insertQuery, (
            data['title'],
            data['price'],
            data['rating'],
            data['review_count'],
            data['sentimental_analysis']
        ))
        connection.commit()
        print(f"Rows affected: {cursor.rowcount}")
    except mysql.connector.Error as err:
        print(f"Error: {err}")

UrlList = []

def get_amazon_product_link(product_name):
    search_url = f"https://www.amazon.com/s?k={product_name.replace(' ', '+')}"
    response = requests.get(search_url, headers=HEADERS)
    soup = BeautifulSoup(response.text, 'html.parser')
    first_product = soup.find('a', {'class':'a-link-normal s-line-clamp-2 s-link-style a-text-normal'})
    if first_product:
        product_link = "https://www.amazon.com" + first_product['href']
        UrlList.append(product_link)
        return product_link
    else:
        print("I am in else")
        return None

def fetch_product_names_from_customer_data():
    product_names = set()
    try:
        cursor.execute("SELECT DISTINCT product_name FROM customer_data WHERE product_name IS NOT NULL AND product_name != ''")
        results = cursor.fetchall()
        for row in results:
            product_names.add(row[0])
    except mysql.connector.Error as err:
        print(f"Error fetching product names: {err}")
    return list(product_names)

def fun(soup, max_comments=1000):
    comments = []
    comment_blocks = soup.find_all("span", {"data-hook": "review-body"}, limit=max_comments)
    for comment in comment_blocks:
        text = comment.get_text(strip=True)
        if text:
            comments.append(text)
    return comments

def insert_top_comments_to_db(title, comments):
    from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
    sia = SentimentIntensityAnalyzer()

    top_3_comments = comments[:3]
    for comment in top_3_comments:
        score = sia.polarity_scores(comment)['compound']
        if score >= 0.6:
            label = "positive"
        elif score <= 0.2:
            label = "negative"
        else:
            label = "neutral"

        insert_query = """
        INSERT INTO review_comments (product_title, comment, sentiment_score, sentiment_label)
        VALUES (%s, %s, %s, %s)
        """
        try:
            cursor.execute(insert_query, (title, comment, score, label))
            connection.commit()
        except mysql.connector.Error as err:
            print(f"Error inserting comment: {err}")


def analyze_sentiment_list(comments):
    total_score = 0
    count = 0
    for comment in comments:
        score = sia.polarity_scores(comment)
        total_score += score['compound']
        count += 1

    avg_score = total_score / count if count > 0 else 0
    sentiment_tag = (
        "positive" if avg_score >= 0.6 else
        "negative" if avg_score <= 0.2 else
        "neutral"
    )
    print(f"Sentiment tag: {sentiment_tag} | Avg score: {avg_score:.3f}")
    return sentiment_tag, round(avg_score, 3)

# Run the script
clearTable()
product_names = fetch_product_names_from_customer_data()
print(product_names)
for pname in product_names:
    get_amazon_product_link(pname)
print(UrlList)
iterateOverURLS(UrlList)

Table cleared successfully.
['Iphone 15 pro', 'Speaker sub', 'Macbook Air', 'Wireless Mouse', 'Perfume, Chain', 'Under Armour Bag', 'Wireless Portable Keyboard, Mouse', 'HP 24 pro, Portable SSD 1TB', 'Apple iPhone 15 pro, Airpods pro, MagSafe', 'Armani Exchange', 'Bracelet', 'HP Zenbook Laptop', 'HP Lazerjet printer', 'Marc Jacobs', 'Decathlon', 'Samsung S24 ultra', 'HDMI Cable, Optical fiber', 'Victoria Secret', 'Hp Zenbook pro', 'Vacuum Cleaner', 'LED Moniter', 'Express wallet, Express clothing', 'Oneplus 7 pro', 'Marks and Spencers', 'Dyson', 'Piano Black Guitar', 'MacBook Pro m2', 'Coach', 'Fitbit, MagSafe', 'Dyson Hairwrap', 'MK Wallet', 'Ck belt', 'Refrigerator', 'iWatch Series 9', 'Zara', 'Keyboard', 'BOSE', 'Bicycle', 'Phone Case', 'iPhone 14 pro', 'Benz Accessories', 'iWatch series 7', 'MK', 'Pendrive', 'Fitbit', 'BOSE Speaker', 'Recliner', 'OTG', 'Iphone 16 pro max, MagSafe Charger', 'Air frier', 'iWatch Ultra, Airpods', 'Prada clothing']
I am in else
I am in else
I am in els